In [ ]:
import os
import shutil
import opendatasets as od
import pandas as pd
from IPython.display import display
from database.sql_connector import SqlConnector

In [ ]:
sql_con = SqlConnector()

## Disney Data 1

In [ ]:
kaggle_user = "prateekmaj21"
folder_name = "disney-movies"

In [ ]:
od.download(f"https://www.kaggle.com/{kaggle_user}/{folder_name}", force=True)

In [ ]:
file = os.listdir(folder_name)[0]
df1 = pd.read_csv(os.path.join(folder_name,file), encoding="ISO-8859-1")
df1.rename(columns={'movie_title':'Title', 'release_date':'Release_Date', 'genre':'Genre',
                    'mpaa_rating':'MPAA_Rating', 'total_gross':'Total_Gross', 'inflation_adjusted_gross':'Inflation_Adjusted_Gross'}, inplace=True)

### Pre-process release date column 

In [ ]:
df1['Release_Date'] = pd.to_datetime(df1['Release_Date'], format="%Y-%m-%d")

### Remove file 

In [ ]:
shutil.rmtree(folder_name)

### Ingest data 

In [ ]:
sql_con.run_query("""DELETE FROM KAGGLE.DISNEY.DATA""")
sql_con.ingest_dataframe(df1, schema='DISNEY', table='DATA')